In [1]:
import glob
import jsonlines
from pathlib import Path
import natsort
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import subprocess
from tqdm import tqdm 

In [20]:
annotations_dir = "/media/data/warp_data/marine_detection/imerit/whoi-rsi-fish-detection-datasets-22122023"
images_dir = "/media/data/warp_data/marine_detection/imerit/whoi-rsi-fish-detection-datasets-22122023"
manifest_path = "/media/data/warp_data/marine_detection/imerit/whoi-rsi-fish-detection-datasets-22122023/28102023_manifest.json"

In [21]:
annotation_files = glob.glob(f"{annotations_dir}/annotations/consolidated-annotation/output/*/*.json")
annotation_files = natsort.natsorted(annotation_files)

In [22]:
# Parse manifest
ordered_video_dirs = []
replace_root = "s3://whoi-rsi-fish-detection/datasets/imerit_26102023_clips/"
with jsonlines.open(manifest_path) as f:
    for video_num, video_json in enumerate(f):
        video_path = Path(video_json['source-ref'].replace(replace_root, ""))
        ordered_video_dirs.append(video_path.parent)

In [25]:
# Generate videos from frames
dry_run = False
overwrite_existing = True

for aws_id, video_dir in enumerate(tqdm(ordered_video_dirs)):
    framerate = 3
    input_glob = str(Path(images_dir) / video_dir) + "/*.png"
        
    #output_path = str(Path(images_dir) / video_dir.parent / video_dir.stem) + ".mp4"
    output_path = str(Path("/media/data/warp_data/marine_detection/imerit/") / "imerit_26102023_3fps_clips" / video_dir.stem) + f"_aws{aws_id}.mp4"
    
    if not dry_run:
        ffmpeg_cmd = ["ffmpeg", "-hide_banner", 
                                    "-loglevel", "error", 
                                    "-framerate", str(framerate), 
                                    "-pattern_type", "glob", 
                                    "-i", input_glob, 
                                    "-c:v", "libx264", 
                                    "-pix_fmt", "yuv420p", 
                                    output_path]
        if overwrite_existing:
            ffmpeg_cmd.append("-y")
        subprocess.call(ffmpeg_cmd)
    else:
        print("input: ", input_glob, " || output: ", output_path)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 163/163 [00:07<00:00, 22.89it/s]
